In [ ]:
#hide
from fastbook import *
from fastai.vision.widgets import *

In [ ]:
key = os.environ.get('AZURE_SEARCH_KEY', '07467503d3b346b58d48307eab4d69d9')

In [ ]:
search_images_bing

In [ ]:
results = search_images_bing(key, 'Toxicodendron Radicans')
ims = results.attrgot('content_url')
len(ims)

In [ ]:
ims = ['http://climbers.lsa.umich.edu/wp-content/uploads/2013/08/Toxicodendron-radicans-leaves-good1.jpg']

In [ ]:
dest ='images/Toxicodendron-radicans-leaves-good1.jpg'
download_url(ims[0], dest)

In [ ]:
im = Image.open(dest)
im.to_thumb(128,128)

In [ ]:
toxicodendron_types = 'Toxicodendron Radicans','Toxicodendron Diversilobum','Toxicodendron Vernix'
path = Path('toxicodendrons')

In [ ]:
if not path.exists():
    path.mkdir()
    for o in toxicodendron_types:
        dest = (path/o)
        dest.mkdir(exist_ok=True)
        results = search_images_bing(key, f'{o} toxicodendron')
        download_images(dest, urls=results.attrgot('content_url'))

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
toxicodendrons = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = toxicodendrons.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
toxicodendrons = toxicodendrons.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = toxicodendrons.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
toxicodendrons = toxicodendrons.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = toxicodendrons.dataloaders(path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
toxicodendrons  = toxicodendrons .new(item_tfms=RandomResizedCrop(128, min_scale=0.3))
dls = toxicodendrons .dataloaders(path)
dls.train.show_batch(max_n=4, nrows=1, unique=True)

In [ ]:
toxicodendrons = toxicodendrons.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = toxicodendrons.dataloaders(path)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
toxicodendrons = toxicodendrons.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = toxicodendrons.dataloaders(path)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
#hide_output
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('images/Toxicodendron-radicans-leaves-good1.jpg')

In [ ]:
learn_inf.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
btn_upload = SimpleNamespace(data = ['images/Toxicodendron-radicans-leaves-good1.jpg'])

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()

In [ ]:
VBox([widgets.Label('Select Your Posion!'), 
      btn_upload, btn_run, out_pl, lbl_pred])

In [ ]:
!pip install voila
!jupyter serverextension enable voila --sys-prefix
!--use-feature=2020-resolver